# 使用selenium和bs4爬虫

## 1.selenium的使用

import os,time
from selenium import webdriver
from bs4 import BeautifulSoup
chromedriver = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chromedriver.exe"#chrome所在驱动
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get('http://user.100xuexi.com/MyDeal/myEBookPurchase.aspx')
time.sleep(2)

driver.find_element_by_id('username').send_keys('18775325832')#

driver.find_element_by_id('txtVerify').clear()#验证码
driver.find_element_by_id('txtVerify').send_keys('69')#验证码

#driver.find_element_by_xpath('//*[@id="conPurchaseA1"]/table/tbody/tr[3]/td[2]/a').click();
driver.find_element_by_xpath('//*[@id="regNow"]').click();

driver.find_element_by_xpath('//*[@id="PurchaseA2"]').click();

#接下来会有新的窗口打开，获取所有窗口句柄
windows = driver.window_handles
driver.switch_to.window(windows[-2])

driver.find_element_by_xpath('//*[@id="lefter"]/div/div[1]/a').click();
#/PracticeCenter/Chapter/Index?TQuestionPlanID=2997&groupName=&code=&tb_l_PaperQuePlanID=86215&Model=chapter&TypeMenuFlag=2
#/PracticeCenter/Chapter/Index?TQuestionPlanID=2997&groupName=&code=&tb_l_PaperQuePlanID=86216&Model=chapter&TypeMenuFlag=2
#item表示章，li后的数字表示节

#driver.find_element_by_xpath('//*[@id="item01"]/li[2]/a').click();

## 2.解析成html

In [2150]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

## 3.html文件解析

### 1.解析问题

In [2151]:
import re

In [2152]:
baseurl_item = u'7.6 设计年径流'
single_baseurl = u'注册土木工程师anki\\专业基础\\'+ baseurl_item+ u'\\'

In [2153]:
single_baseurl

'注册土木工程师anki\\专业基础\\7.6 设计年径流\\'

In [2154]:
content_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [2155]:
#soup.find('div',class_='ItemBotCompose').decompose()

In [2156]:
# replace with `soup.findAll` if you are using BeautifulSoup3
for div in soup.find_all("div", {'class':'Item config'}): 
    if len(div.find_all("div", {'class':'ItemBotCompose'}))!=0:
        div.decompose() #将满足调节的 div 项删除

In [2157]:
single_content_all = []

single_content_all = content_all[0].find_all("div",attrs={"class":"question config"})
if len(content_all)>1:
    multiple_content_all = content_all[1].find_all("div",attrs={"class":"question config"})

In [2158]:
len(single_content_all)

46

单选

In [2159]:
single_question = []

for i in range(len(single_content_all)):
    single_contents = single_content_all[i].dt.contents
    #print(contents_single)
    single_s = ''
    for j in single_contents:#单选
        single_s = single_s+str(j)
    single_question.append(re.sub('[\n]','',single_s))

In [2160]:
print(len(single_question))

46


保存数据

In [2161]:
#question_file = open('question_file.txt', 'w')
import codecs

single_folder = os.path.exists(single_baseurl)
#multiple_folder = os.path.exists(multiple_baseurl)
if not single_folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
    os.makedirs(single_baseurl)
                
#if not multiple_folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
#    os.makedirs(multiple_baseurl)
                
single_question_file = codecs.open(single_baseurl + 'single_question.txt', 'w','utf-8')
#multiple_question_file = codecs.open(multiple_baseurl + 'multiple_question.txt', 'w','utf-8')

for item in single_question:
    single_question_file.write("%s\r\n" % item)
    
#for item in multiple_question:
#    multiple_question_file.write("%s\r\n" % item)
    
single_question_file.close()
#multiple_question_file.close()

### 2.解析选项

In [2162]:
option_all = soup.find_all("div",attrs={"class":"nodeItem"})

区别单选和多选

In [2163]:
single_option_all = option_all[0].find_all("div",attrs={"class":"question config"})
if len(option_all)>1:
    multiple_option_all = option_all[1].find_all("div",attrs={"class":"question config"})
else:
    multiple_option_all = []

In [2164]:
len(single_option_all)

46

In [2165]:
"""
_single_option_a = ''
for i in single_option_all[11].table.tbody.find_all("label")[2].children:
    if 'input' in str(i):
        pass
    else:
        if 'A.' in str(i):
            _single_option_a = _single_option_a + re.sub('[\r|\n|\t|\\s]', '',str(i))
        else:
            if 'img' not in str(i):
                _single_option_a = _single_option_a + re.sub('[\r|\n|\t|\\s]', '',str(i))
            else:
                _single_option_a = _single_option_a + str(i)
#single_option_a.append(_single_option_a)
_single_option_a
"""

'\n_single_option_a = \'\'\nfor i in single_option_all[11].table.tbody.find_all("label")[2].children:\n    if \'input\' in str(i):\n        pass\n    else:\n        if \'A.\' in str(i):\n            _single_option_a = _single_option_a + re.sub(\'[\r|\n|\t|\\s]\', \'\',str(i))\n        else:\n            if \'img\' not in str(i):\n                _single_option_a = _single_option_a + re.sub(\'[\r|\n|\t|\\s]\', \'\',str(i))\n            else:\n                _single_option_a = _single_option_a + str(i)\n#single_option_a.append(_single_option_a)\n_single_option_a\n'

单选

In [2166]:
single_option_a = []
single_option_b = []
single_option_c = []
single_option_d = []
for option_i in range(len(single_option_all)):
    
    print(option_i)
    _single_option_a = ''
    _single_option_b = ''
    _single_option_c = ''
    _single_option_d = ''
    
    for i in single_option_all[option_i].table.tbody.find_all("label")[0].children:
        if 'input' in str(i):
            pass
        else:
            if 'img' not in str(i):
                _single_option_a = _single_option_a + re.sub('[\r|\n|\t|\\s]', '',str(i))
            else:
                _single_option_a = _single_option_a + str(i)
    single_option_a.append(_single_option_a)
    
    for i in single_option_all[option_i].table.tbody.find_all("label")[1].children:
        if 'input' in str(i):
            pass
        else:
            if 'img' not in str(i):
                _single_option_b = _single_option_b + re.sub('[\r|\n|\t|\\s]', '',str(i))
            else:
                _single_option_b = _single_option_b + str(i)
    single_option_b.append(_single_option_b)
    
    if len(single_option_all[option_i].table.tbody.find_all("label")) == 2:
        single_option_c.append('C．'+'null')
    else:
        for i in single_option_all[option_i].table.tbody.find_all("label")[2].children:
            if 'input' in str(i):
                pass
            else:
                if 'img' not in str(i):
                    _single_option_c = _single_option_c + re.sub('[\r|\n|\t|\\s]', '',str(i))
                else:
                    _single_option_c = _single_option_c + str(i)
        single_option_c.append(_single_option_c)
    
    if len(single_option_all[option_i].table.tbody.find_all("label"))!=4:
        single_option_d.append('D．'+'null')
    else:
        for i in single_option_all[option_i].table.tbody.find_all("label")[3].children:
            if 'input' in str(i):
                pass
            else:
                if 'img' not in str(i):
                    _single_option_d = _single_option_d + re.sub('[\r|\n|\t|\\s]', '',str(i))
                else:
                    _single_option_d = _single_option_d + str(i)
        single_option_d.append(_single_option_d)
    
    
    """
    if single_option_all[option_i].table.tbody.find_all("label")[0].find_all("img") != []:
        #print(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].find_all("img")))
        single_option_a.append('A．'+str(single_option_all[option_i].table.tbody.find_all("label")[0].find_all("img")[0]))
    else:
        single_option_a.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].get_text()))
        
    if single_option_all[option_i].table.tbody.find_all("label")[1].find_all("img") != []:
        #print(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].find_all("img")))
        single_option_b.append('B．'+str(single_option_all[option_i].table.tbody.find_all("label")[1].find_all("img")[0]))
    else:
        single_option_b.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[1].get_text()))
    
    if single_option_all[option_i].table.tbody.find_all("label")[2].find_all("img") != []:
        #print(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].find_all("img")))
        single_option_c.append('C．'+str(single_option_all[option_i].table.tbody.find_all("label")[2].find_all("img")[0]))
    else:
        single_option_c.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[2].get_text()))
    
    if len(single_option_all[option_i].table.tbody.find_all("label"))!=4:
        single_option_d.append('D．'+'null')
    else:
        if single_option_all[option_i].table.tbody.find_all("label")[3].find_all("img") != []:
            #print(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].find_all("img")))
            single_option_d.append('D．'+str(single_option_all[option_i].table.tbody.find_all("label")[3].find_all("img")[0]))
        else:
            single_option_d.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[3].get_text()))
    """
    
    #single_option_b.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[1].get_text()))
    #single_option_c.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[2].get_text()))
    #single_option_d.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[3].get_text()))
single_option_d

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


['D．样本分布参数与总体分布参数的接近程度',
 'D．年最大的洪水',
 'D．可行性、代表性、一致性',
 'D．考虑安全',
 'D．设计值偏于安全原则',
 'D．峰型特征',
 'D．涨洪历时',
 'D．用相关分析法推求',
 'D．固定时段最小值法',
 'D．由历史洪水调查考证确定',
 'D．经验频率和统计参数的计算',
 'D．<img height="27" src="http://admintk.100xuexi.com/UpLoadImage/2013-05-07/0a13eaf4-ef0b-4813-b2dd-7f0604da4e56.png" width="81"/>，C<sub>SN=</sub>C<sub>Sn</sub>',
 'D．同一测站，随历时增长，设计洪量增值增大，',
 'D．50%',
 'D．经验确定',
 'D．不能肯定',
 'D．不能肯定',
 'D．必须是单峰',
 'D．频率控制的倍比',
 'D．峰和量都不等于设计值',
 'D．峰和量都不等于设计值',
 'D．峰和量都不等于设计值',
 'D．典型洪水历时长，洪量较大',
 'D．峰、短历时洪量、长历时洪量',
 'D．典型洪水过程线的变化趋势',
 'D．成果鉴定验收',
 'D．本流域年降水资料',
 'D．检查洪水的大小',
 'D．检查系列的长短',
 'D．检查洪水的大小',
 'D．100',
 'D．4～5',
 'D．参证变量长系列与设计变量非同期的参证变量系列',
 'D．年径流的均值、C<sub>v</sub>、最大值和最小值',
 'D．经验分析',
 'D．水文调查法',
 'D．水文调查法',
 'D．25%',
 'D．年径流量',
 'D．相同',
 'D．减去',
 'D．不宜',
 'D．与设计断面距离较近',
 'D．大暴雨资料容易收集',
 'D．与大洪水时段对应的时段年最大值法',
 'D．可以用流域中心站设计点雨量乘以点面折算系数代表']

In [2167]:
single_option_a[0]

'A．是否有特大洪水'

保存单选选项

In [2168]:
import codecs

single_option_a_file = codecs.open(single_baseurl + 'single_option_a.txt', 'w','utf-8')
single_option_b_file = codecs.open(single_baseurl + 'single_option_b.txt', 'w','utf-8')
single_option_c_file = codecs.open(single_baseurl + 'single_option_c.txt', 'w','utf-8')
single_option_d_file = codecs.open(single_baseurl + 'single_option_d.txt', 'w','utf-8')

for item in single_option_a:
    single_option_a_file.write("%s\r\n" % item)
for item in single_option_b:
    single_option_b_file.write("%s\r\n" % item)
    
for item in single_option_c:
    single_option_c_file.write("%s\r\n" % item)
    
for item in single_option_d:
    single_option_d_file.write("%s\r\n" % item)

single_option_a_file.close()
single_option_b_file.close()
single_option_c_file.close()
single_option_d_file.close()

### 解析答案

In [2169]:
anwser_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [2170]:
single_anwser_all = anwser_all[0].find_all("div",attrs={"class":"Item config"}) #单选
if len(anwser_all)>1:
    multiple_anwser_all = anwser_all[1].find_all("div",attrs={"class":"Item config"}) #多选
else:
    multiple_anwser_all = []

In [2171]:
len(single_anwser_all)

46

In [2172]:
single_answer_right = []
#multiple_answer_right = []

for answer_i in range(len(single_anwser_all)):
    #print(anwser_all[25].find("div",attrs={"class":"config answer"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}).get_text())
    single_answer_right.append(single_anwser_all[answer_i].find("div",attrs={"class":"anser Con"}).string)

len(single_answer_right)

46

In [2173]:
len(single_anwser_all)

46

In [2174]:
import codecs

single_anwser_file = codecs.open(single_baseurl + 'single_anwser.txt', 'w','utf-8')
#multiple_anwser_file = codecs.open(multiple_baseurl + 'multiple_anwser.txt', 'w','utf-8')

#for item in multiple_answer_right:
#    multiple_anwser_file.write("%s\r\n" % item)
    
single_anwser_file.close()
#multiple_anwser_file.close()

In [2175]:
#print(re.sub('[\r|\n|\t|\\s]', '', anwser_1))

### 解析解析

In [2176]:
analysis_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [2177]:
single_analysis_all = analysis_all[0].find_all("div",attrs={"class":"Item config"}) #单选
if len(analysis_all)>1:
    multiple_analysis_all = analysis_all[1].find_all("div",attrs={"class":"Item config"}) #多选
else:
    multiple_analysis_all = []
len(single_analysis_all)

46

In [2178]:
single_analysis = []
#multiple_analysis = []

for analysis_i in range(len(single_analysis_all)):
    print(single_analysis_all[analysis_i].find("div",attrs={"class":"analysits Con"}))
    print('\n')


<div class="analysits Con">通过统计分析来估计某种随机事件的概率特性，必须要有一个好的样本作为基础。样本资料的质量主要反映在是否满足下列三方面的要求：①资料应具有充分的可靠性；②资料的基础应具有一致性；③样本系列应具有充分的代表性。代表性是工程技术人员针对具体一个样本，说明它与总体之间误差情况的概念。经验分布Fn（x）与总体分布F（x），两者之间的差异愈小、愈接近，即说明样本的代表性愈好，反之则愈差。</div>


<div class="analysits Con">设计洪水和设计任何水工建筑物时，都需要选定某一量级的洪水作为设计依据，以便确定工程的规模。现行的方法是选择某一累积频率（如1%、0.1%）的洪水作为设计依据或标准，可视工程大小与重要程度选定不同的累积频率。设计标准确定后，按标准推求的洪水，称为设计洪水。</div>


<div class="analysits Con">资料的审查是指在频率计算之前，须进行枯水资料的可靠性、一致性和代表性的分析审查，包括相关流域的自然地理概况、流域河道特征、有明显人类活动影响的工程措施、水文气象资料以及前人分析的有关成果。</div>


<div class="analysits Con">在设计年径流的分析计算中，对年径流系列较长且较完整的资料，可直接据以进行频率分析，确定所需的设计年径流量，对短缺资料的流域，应尽量设法延长其径流系列，或用间接方法，经过合理的论证和修正、移用参证流域的设计成果，当实测资料的年数不足20年，或有20年但资料系列不连续或代表性不足，一般都要进行插补延长，其目的是增加系列的代表性。年径流系列的代表性，是指该样本对年径流总体的接近程度，如接近程度较高，则系列的代表性较好，频率分析成果的精度较高，反之较低。</div>


<div class="analysits Con">样本系列各项的经验频率初算确定之后，在几率格纸上绘出经验频率点据的位置，目估绘出一条平滑的曲线，称为经验频率曲线。我国一直采用皮尔逊型一Ⅲ型曲线，作为洪水频率计算的依据，检查曲线与经验频率点据的配合情况。若配合得不好，应适当调整参数值，直到配合较好为止。</div>


<div class="analysits Con">洪水过程通常用三个要素描述，即洪峰流量Q<sub>m</sub>

In [2179]:
#print(multiple_analysis_all[0].find("div",attrs={"class":"analysits Con"}))

In [2180]:
single_analay = []
#multiple_analay = []
for i in range(len(single_analysis_all)):
    single_analay_item = single_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents
    #print(analay_single)
    single_analay.append(single_analay_item)
    #print(analay_single)
"""    
for i in range(len(multiple_analysis_all)):
    multiple_analay_item = multiple_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents
    #print(analay_single)
    multiple_analay.append(multiple_analay_item)
    #print(analay_single)
"""

'    \nfor i in range(len(multiple_analysis_all)):\n    multiple_analay_item = multiple_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents\n    #print(analay_single)\n    multiple_analay.append(multiple_analay_item)\n    #print(analay_single)\n'

In [2181]:
single_analay_final = []
for item in single_analay:
    #analysis_file.write("%s" % item)
    single_s = ''
    for i in item:
        single_s = single_s + str(i)
    single_analay_final.append(single_s)
    #analysis_file.write("%s\n" % s)
#analysis_file.close()
"""
multiple_analay_final = []
for item in multiple_analay:
    #analysis_file.write("%s" % item)
    multiple_s = ''
    for i in item:
        multiple_s = multiple_s + str(i)
    multiple_analay_final.append(multiple_s)
    #analysis_file.write("%s\n" % s)
#analysis_file.close()
"""

'\nmultiple_analay_final = []\nfor item in multiple_analay:\n    #analysis_file.write("%s" % item)\n    multiple_s = \'\'\n    for i in item:\n        multiple_s = multiple_s + str(i)\n    multiple_analay_final.append(multiple_s)\n    #analysis_file.write("%s\n" % s)\n#analysis_file.close()\n'

In [2182]:
import codecs
single_analysis_file = codecs.open(single_baseurl + 'single_analysis_file.txt', 'w','utf-8')

for s in single_analay_final:
    print(s)
    print("\n")
    single_analysis_file.write("%s\r\n" % s)
""" 
multiple_analysis_file = codecs.open(multiple_baseurl + 'multiple_analysis_file.txt', 'w','utf-8')

for s in multiple_analay_final:
    print(s)
    print("\n")
    multiple_analysis_file.write("%s\r\n" % s)    
"""    
single_analysis_file.close()
#multiple_analysis_file.close()

通过统计分析来估计某种随机事件的概率特性，必须要有一个好的样本作为基础。样本资料的质量主要反映在是否满足下列三方面的要求：①资料应具有充分的可靠性；②资料的基础应具有一致性；③样本系列应具有充分的代表性。代表性是工程技术人员针对具体一个样本，说明它与总体之间误差情况的概念。经验分布Fn（x）与总体分布F（x），两者之间的差异愈小、愈接近，即说明样本的代表性愈好，反之则愈差。


设计洪水和设计任何水工建筑物时，都需要选定某一量级的洪水作为设计依据，以便确定工程的规模。现行的方法是选择某一累积频率（如1%、0.1%）的洪水作为设计依据或标准，可视工程大小与重要程度选定不同的累积频率。设计标准确定后，按标准推求的洪水，称为设计洪水。


资料的审查是指在频率计算之前，须进行枯水资料的可靠性、一致性和代表性的分析审查，包括相关流域的自然地理概况、流域河道特征、有明显人类活动影响的工程措施、水文气象资料以及前人分析的有关成果。


在设计年径流的分析计算中，对年径流系列较长且较完整的资料，可直接据以进行频率分析，确定所需的设计年径流量，对短缺资料的流域，应尽量设法延长其径流系列，或用间接方法，经过合理的论证和修正、移用参证流域的设计成果，当实测资料的年数不足20年，或有20年但资料系列不连续或代表性不足，一般都要进行插补延长，其目的是增加系列的代表性。年径流系列的代表性，是指该样本对年径流总体的接近程度，如接近程度较高，则系列的代表性较好，频率分析成果的精度较高，反之较低。


样本系列各项的经验频率初算确定之后，在几率格纸上绘出经验频率点据的位置，目估绘出一条平滑的曲线，称为经验频率曲线。我国一直采用皮尔逊型一Ⅲ型曲线，作为洪水频率计算的依据，检查曲线与经验频率点据的配合情况。若配合得不好，应适当调整参数值，直到配合较好为止。


洪水过程通常用三个要素描述，即洪峰流量Q<sub>m</sub>、一次洪水过程总量W、洪水历时（由涨水历时t<sub>1</sub>与退水历时t<sub>2</sub>相加求得）。设计洪水计算的主要包括设计洪峰流量、不同时段（最大一、三、五、七、十五日……）的设计洪水总量和设计洪水过程等三项内容。


设计洪水计算的内容主要包括设计洪峰流量、不同时段（最大一、三、五、七、十五日……）的设计洪水总量和设计洪水过程等。工程特点不同，需要计算的设计洪

单选选项拼接

In [2183]:
len(single_option_a)

46

In [2184]:
single_data = []
for i in range(len(single_option_a)):
    single_data_string = single_question[i].replace('（　　）','{{c1::（）}}') + "	"+ single_option_a[i].strip().replace('A．','')+ '<br>' + single_option_b[i].strip().replace('B．','')+ '<br>'+ single_option_c[i].strip().replace('C．','')+ '<br>'+ single_option_d[i].replace('D．','')+ "	"+ single_answer_right[i]+ "	"+ single_analay_final[i]
    single_data.append(single_data_string)
data_single = codecs.open(single_baseurl + 'data_single.txt', 'w','utf-8')

for i in range(len(single_data)):
    data_single.write(single_data[i]+"\r\n")
data_single.close()

In [2185]:
len(single_data )

46

多选选项拼接

In [2186]:
#len(multiple_data)